# NGY API Reverse engineering

1. https://app.g-ny.org/
1. Ctrl+Shift+I
1. ???
1. profit

In [53]:
import requests
import json
import pandas as pd
import numpy as np
from pytz import timezone
tz = timezone('Europe/Paris')

from ipyleaflet import Map, GeoJSON, Circle, CircleMarker
from geopy.geocoders import Nominatim
geolocator = Nominatim()
from fuzzywuzzy import process
import ipywidgets
from IPython.display import display, clear_output, HTML

In [2]:
NGY_URL = "https://app.g-ny.org/thrift/js/stops"

HEADER = {
    "Host": "app.g-ny.org",
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:54.0) Gecko/20100101 Firefox/54.0",
    "Accept": "*/*",
    "Accept-Language": "en-US,fr-FR;q=0.7,en;q=0.3",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://app.g-ny.org/",
    "Content-Type": "text/plain;charset=UTF-8"
}

# Fetching a list of stops and lines

These shouldn't be requested often and actually could be fetched and updated from the [Open Data Grand Nancy](http://opendata.grandnancy.eu/jeux-de-donnees/detail-dune-fiche-de-donnees/?tx_icsoddatastore_pi1%5Bkeywords%5D=&tx_icsoddatastore_pi1%5Bfileformat%5D%5B0%5D=13&tx_icsoddatastore_pi1%5Buid%5D=108&tx_icsoddatastore_pi1%5BreturnID%5D=447) website.

In [3]:
def getGTStops():
    """ Get the full list of tramway and bus stops
    """
    
    payload = json.dumps(
        [
            1, "getGTStops", 1, 0,
            {"1": {"i32": 0}}
        ],
        separators=(",", ":")
        )
    
    resp = requests.post(NGY_URL, headers=HEADER, data=payload)

    stops = resp.json()[4]["0"]["lst"][2:]
    
    titlelize = lambda lst: [elem.title() for elem in lst]
    prettify = lambda fugly: " ".join(titlelize(fugly.split()[:-1]))
    
    cleaner_stops = [
        {
            "id": stop["2"]["str"],
            "name": stop["3"]["str"],
            "pretty_name": prettify(stop["3"]["str"]), # a dataset normally shouldn't hold processed data
                                                       # this is here just for convenience and out of lazy design
            "lat": float(stop["4"]["dbl"]),
            "lon": float(stop["5"]["dbl"])
        }
        for stop in stops
    ]

    return pd.DataFrame(cleaner_stops)

In [58]:
stops = getGTStops()
stops

,id,lat,lon,name,pretty_name
0,395,48.656199,6.265249,18 JUIN (ART-SUR-MEURTHE),18 Juin
1,1342,48.701448,6.176614,3 MAISONS (NANCY),3 Maisons
2,1518,48.656217,6.221625,5 FONTAINES (LANEUVEVILLE-DEVANT-NANCY),5 Fontaines
3,1416,48.705450,6.230390,69 EME RI (ESSEY-LES-NANCY),69 Eme Ri
4,687,48.678240,6.197830,ACHILLE LEVY (NANCY),Achille Levy
5,1625,48.657219,6.134535,AIR LORRAINE (VILLERS-LES-NANCY),Air Lorraine
6,159,48.614699,6.166468,ALBATRE (LUDRES),Albatre
7,1320,48.672959,6.147647,ALBERT 1ER (VILLERS-LES-NANCY),Albert 1Er
8,493,48.704441,6.192353,ALEXANDRE 1ER (MALZEVILLE),Alexandre 1Er
9,1216,48.702020,6.164100,ALIX LE CLERC (NANCY),Alix Le Clerc


In [72]:
stops[stops["name"] == process.extractOne("velodrome", stops["name"].values)[0]]

,id,lat,lon,name,pretty_name
506,1052,48.666274,6.167195,VELODROME (VANDOEUVRE-LES-NANCY),Velodrome


In [78]:
def get_stops_around_place(addr, d=1000):
    
    stops = getGTStops()
    
    place = geolocator.geocode(addr)
    
    if not place:
        return "Place not found"
    
    m = Map(center=[place.latitude, place.longitude])
    
    cm = CircleMarker(location=m.center, radius=5, weight=2,
                      color='#F00', opacity=1.0, fill_opacity=1.0,
                      fill_color='#F00')
    m.add_layer(cm)
    
    c = Circle(location=m.center, radius=d, weight=1,
                color='#F00', opacity=1.0, fill_opacity=0.3,
                fill_color='#F00')
    m.add_layer(c)
    
    # http://janmatuschek.de/LatitudeLongitudeBoundingCoordinates
    
    r = d / (6371 * 1000)
    latT = np.arcsin(np.sin(np.deg2rad(place.latitude)) / np.cos(r))
    dlon = np.arcsin(np.sin(r) / np.cos(np.deg2rad(place.latitude)))
    
    latmin = np.deg2rad(place.latitude) - r
    latmax = np.deg2rad(place.latitude) + r
    lonmin = np.deg2rad(place.longitude) - dlon
    lonmax = np.deg2rad(place.longitude) + dlon

    geojsondata = {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "geometry": {
                    "type": "Point",
                    "coordinates": [
                        stop[1]["lon"],
                        stop[1]["lat"]
                    ]
                },
                "properties": {
                    "name": stop[1]["pretty_name"],
                    "id": stop[1]["id"]
                }
            }
            for stop in stops[
                stops["lat"].between(np.rad2deg(latmin), np.rad2deg(latmax))
                & stops["lon"].between(np.rad2deg(lonmin), np.rad2deg(lonmax))
                & (np.arccos(np.sin(np.deg2rad(place.latitude)) * np.sin(np.deg2rad(stops["lat"]))
                           + np.cos(np.deg2rad(place.latitude)) * np.cos(np.deg2rad(stops["lat"]))
                           * np.cos(np.deg2rad(stops["lon"]) - (np.deg2rad(place.longitude)))) <= r)
            ].iterrows()
        ]
    }

    g = GeoJSON(data=geojsondata)
    m.add_layer(g)
    return m

get_stops_around_place("place stanislas, nancy, France", d=500)

In [6]:
def getGTLines():
    """ Get the full list of tramway and bus lines
    """
    
    payload = json.dumps(
        [
            1, "getGTLines", 1, 0,
            {"1": {"i32": 0}}
        ],
        separators=(",", ":")
        )
    
    resp = requests.post(NGY_URL, headers=HEADER, data=payload)
    
    lines = resp.json()[4]["0"]["lst"][2:]
    
    cleaner_lines = [
        {
            "id": line["1"]["str"],
            "name": line["2"]["str"],
            "desc": line["3"]["str"],
            "bg_color": line["5"]["str"],
            "fg_color": line["6"]["str"]
        }
        for line in lines
    ]

    return pd.DataFrame(cleaner_lines)

In [7]:
lines = getGTLines()
lines

,bg_color,desc,fg_color,id,name
0,E0001A,Vandoeuvre CHU Brabois - Essey Mouzimpré,FFFFFF,101,1
1,0087C7,Laxou Plateau de Haye - Laneuveville Centre,FFFFFF,102,2
2,008F37,Laxou Provinces - Seichamps Haie Cerlin,FFFFFF,103,3
3,FEDC00,Laxou Champ-le-Boeuf - Vandoeuvre Roberval,000000,104,4
4,DF0077,Laxou Sapinière - Seichamps Haie Cerlin,FFFFFF,105,5
5,B1C800,Villers Clairlieu - Malzeville Savlons,FFFFFF,106,6
6,F19200,Houdemont Porte Sud / Vandoeuvre Roberval - Do...,FFFFFF,107,7
7,00A7E0,Vandoeuvre CHU - Malzéville (Pixerécourt),FFFFFF,108,8
8,804A26,Nancy Tamaris - Saulxures Centre / Forêt,FFFFFF,109,9
9,7F197F,Nancy Pôle de Santé - Jarville Californie,FFFFFF,110,10


# Fetching the stops of a certain line

In [8]:
def getGTStopsFromGTLine(id):
    """ Get the stops of a line given its id
    """
    
    payload = json.dumps(
        [
            1, "getGTStopsFromGTLine", 1, 0,
            {"1": {"str": id}, "2": {"i32": 0}}
        ],
        separators=(",", ":")
        )
    
    resp = requests.post(NGY_URL, headers=HEADER, data=payload)
    
    stops = resp.json()[4]["0"]["lst"][2:]
    
    titlelize = lambda lst: [elem.title() for elem in lst]
    prettify = lambda fugly: " ".join(titlelize(fugly.split()[:-1]))
    
    cleaner_stops = [
        {
            "id": stop["2"]["str"],
            "name": stop["3"]["str"],
            "pretty_name": prettify(stop["3"]["str"]),
            "lat": float(stop["4"]["dbl"]),
            "lon": float(stop["5"]["dbl"])
        }
        for stop in stops
    ]
    
    return pd.DataFrame(cleaner_stops)

In [9]:
getGTStopsFromGTLine(lines[lines["name"] == "8"].id.values[0])

,id,lat,lon,name,pretty_name
0,1625,48.657219,6.134535,AIR LORRAINE (VILLERS-LES-NANCY),Air Lorraine
1,1332,48.672945,6.163896,AVENUE DE BRABOIS (VANDOEUVRE-LES-NANCY),Avenue De Brabois
2,238,48.696362,6.174433,BARON LOUIS (NANCY),Baron Louis
3,1479,48.707455,6.188495,BARRES (MALZEVILLE),Barres
4,335,48.677206,6.160525,BAUDRICOURT (VILLERS-LES-NANCY),Baudricourt
5,774,48.679269,6.161435,BERTIN (NANCY),Bertin
6,1114,48.658241,6.138453,CAMPING (VILLERS-LES-NANCY),Camping
7,1577,48.650923,6.146069,CAMPUS BRABOIS ECOLE D INGENIEURS (VANDOEUVRE-...,Campus Brabois Ecole D Ingenieurs
8,1578,48.650985,6.142301,CAMPUS BRABOIS FACULTE DE MEDECINE (VANDOEUVRE...,Campus Brabois Faculte De Medecine
9,784,48.681910,6.163715,CHARLEMAGNE (NANCY),Charlemagne


# Fetching the lines passing by a certain stop

In [10]:
def getGTLinesFromGTStop(name):
    """ Get the lines passing by a given stop
    """
    
    payload = json.dumps(
        [
            1, "getGTLinesFromGTStop", 1, 0,
            {"1": {"str": name}, "2": {"i32": 0}}
        ],
        separators=(",", ":")
    )
    
    resp = requests.post(NGY_URL, headers=HEADER, data=payload)
    
    lines = resp.json()[4]["0"]["lst"][2:]
    
    cleaner_lines = [
        {
            "id": line["1"]["str"],
            "name": line["2"]["str"],
            "desc": line["3"]["str"],
            "bg_color": line["5"]["str"],
            "fg_color": line["6"]["str"]
        }
        for line in lines
    ]
    
    return pd.DataFrame(cleaner_lines)

In [69]:
getGTLinesFromGTStop(process.extractOne("velodrome", stops["name"].values)[0])

,bg_color,desc,fg_color,id,name
0,E0001A,Vandoeuvre CHU Brabois - Essey Mouzimpré,FFFFFF,101,1
1,7F197F,Nancy Pôle de Santé - Jarville Californie,FFFFFF,110,10
2,A85E21,Laneuveville Gare - Villers Lycée Stanislas,FFFFFF,115,15
3,59AA27,Ludres J.Monod - Villers Lycée Stanislas,FFFFFF,117,17
4,00A9A2,Houdemont les Egrez / Vandoeuvre Monplaisir - ...,FFFFFF,084,E
5,182981,Fléville / Villers Lycée Stanislas,FFFFFF,097,S


# Get the directions of a line from a given stop

In [12]:
def getDestinationsByGTStopsAndGTLines(stop_name, line_id):
    """ Get the directions of a line from a given stop
    """
    
    payload = json.dumps(
        [
            1, "getDestinationsByGTStopsAndGTLines", 1, 0,
            {"1": {"str": stop_name}, "2": {"str": line_id}, "3": {"i32": 0}}
        ],
        separators=(",", ":")
    )
    
    resp = requests.post(NGY_URL, headers=HEADER, data=payload)
    
    destinations = resp.json()[4]["0"]["lst"][2:]
    
    return destinations

In [13]:
stop = process.extractOne("joseph laurent", stops["name"].values)[0]
lines = getGTLinesFromGTStop(stop)
line = lines[lines["name"] == "S"].id.values[0]

getDestinationsByGTStopsAndGTLines(stop, line)

['S FLEVILLE', 'S VILLERS STAN']

# Get Schedules given a stop, the line and its destination

In [14]:
def getSchedulesByGTStopGTLineDestinationsV2(
    stop_name,
    date,
    line_id,
    destination
):
    """ Get Schedules given a stop, the line and its destination
    """
    
    payload = json.dumps(
        [
            1, "getSchedulesByGTStopGTLineDestinationsV2", 1, 0,
            {
                "1": {"str": stop_name},
                "2": {"i64": date},
                "3": {"str": line_id},
                "4": {"str": destination},
                "5": {"i32": 0}
            }
        ],
        separators=(",", ":")
    )
    
    resp = requests.post(NGY_URL, headers=HEADER, data=payload)
    
    horaires = resp.json()[4]["0"]["lst"][2:]
    
    realtime = [
        {
            "time": pd.Timestamp(passage_time),
            "timedelta": pd.Timedelta(passage_delta),
            "real_time": not bool(not_real_time)
        }
        for passage_time, passage_delta, not_real_time in zip(
            horaires[0]["6"]["lst"][2:],
            horaires[0]["8"]["lst"][2:],
            horaires[0]["9"]["lst"][2:]
        )
    ]
    
    theoretical = [
        pd.Timestamp(tstr)
        for tstr in horaires[1]["6"]["lst"][2:]
    ]
    
    return realtime, theoretical

In [15]:
fuzzy_stop_name = "joseph laurent"
line_name = "8"

stop = process.extractOne(fuzzy_stop_name, stops["name"].values)[0]
lines = getGTLinesFromGTStop(stop)
line = lines[lines["name"] == line_name].id.values[0]
dests = getDestinationsByGTStopsAndGTLines(stop, line)
date = int(pd.Timestamp.today(tz).to_pydatetime().date().strftime("%s")) * 1000

getSchedulesByGTStopGTLineDestinationsV2(stop, date, line, dests[1])

([{'real_time': True,
   'time': Timestamp('2017-08-25 16:38:39'),
   'timedelta': Timedelta('0 days 00:04:00')},
  {'real_time': True,
   'time': Timestamp('2017-08-25 17:02:23'),
   'timedelta': Timedelta('0 days 00:28:00')},
  {'real_time': False,
   'time': Timestamp('2017-08-25 17:22:09'),
   'timedelta': Timedelta('0 days 00:48:00')},
  {'real_time': False,
   'time': Timestamp('2017-08-25 17:42:09'),
   'timedelta': Timedelta('0 days 01:08:00')}],
 [Timestamp('2017-08-25 05:44:00'),
  Timestamp('2017-08-25 06:24:00'),
  Timestamp('2017-08-25 07:01:00'),
  Timestamp('2017-08-25 07:25:00'),
  Timestamp('2017-08-25 07:45:00'),
  Timestamp('2017-08-25 08:05:00'),
  Timestamp('2017-08-25 08:27:00'),
  Timestamp('2017-08-25 08:48:00'),
  Timestamp('2017-08-25 09:07:00'),
  Timestamp('2017-08-25 09:27:00'),
  Timestamp('2017-08-25 10:17:00'),
  Timestamp('2017-08-25 10:57:00'),
  Timestamp('2017-08-25 11:37:00'),
  Timestamp('2017-08-25 12:08:00'),
  Timestamp('2017-08-25 12:37:00'),
 

In [16]:
# Setup and display dropdown widgets
# ----------------------------------

stop_select = ipywidgets.Dropdown(
    description="Stop:",
    options=stops["name"].values.tolist()
)

default_lines = getGTLinesFromGTStop(stop_select.value)

line_select = ipywidgets.Dropdown(
    description="Line:",
    options={
        "[" + row[1]["name"] + "]: " + row[1]["desc"]: row[1]["id"]
        for row in default_lines.iterrows()
    }
)

default_dests = getDestinationsByGTStopsAndGTLines(stop_select.value, line_select.value)

destinations_select = ipywidgets.Dropdown(
    description="Destination:",
    options=default_dests
)

date = int(pd.Timestamp.today(tz).to_pydatetime().date().strftime("%s")) * 1000

result = getSchedulesByGTStopGTLineDestinationsV2(
    stop_select.value,
    date,
    line_select.value,
    destinations_select.value
)

display(stop_select)
display(line_select)
display(destinations_select)

if result[0] and result[0][0] and result[0][0]["timedelta"]:
    print("Next passage in ", result[0][0]["timedelta"])
else:
    print("Nothing found")

# Setup the event observers' callbacks
# ------------------------------------

def stop_observer(change):
    stop = change["new"]
    lines = getGTLinesFromGTStop(stop_select.value)
    line_select.options = {
        "[" + row[1]["name"] + "]: " + row[1]["desc"]: row[1]["id"]
        for row in lines.iterrows()
    }
    destinations_select.options = getDestinationsByGTStopsAndGTLines(stop, line_select.value)
    result = getSchedulesByGTStopGTLineDestinationsV2(
        stop_select.value,
        date,
        line_select.value,
        destinations_select.value
    )
    clear_output(wait=True)
    if result[0] and result[0][0] and result[0][0]["timedelta"]:
        print("Next passage in ", result[0][0]["timedelta"])
    else:
        print("Nothing found")

def line_observer(change):
    destinations_select.options = getDestinationsByGTStopsAndGTLines(stop_select.value, change["new"])
    result = getSchedulesByGTStopGTLineDestinationsV2(
        stop_select.value,
        date,
        line_select.value,
        destinations_select.value
    )
    clear_output(wait=True)
    if result[0] and result[0][0] and result[0][0]["timedelta"]:
        print("Next passage in ", result[0][0]["timedelta"])
    else:
        print("Nothing found")

def destination_observer(change):
    result = getSchedulesByGTStopGTLineDestinationsV2(
        stop_select.value,
        date,
        line_select.value,
        change["new"]
    )
    clear_output(wait=True)
    if result[0] and result[0][0] and result[0][0]["timedelta"]:
        print("Next passage in ", result[0][0]["timedelta"])
    else:
        print("Nothing found")

# Register callbacks to the widgets
# ---------------------------------

stop_select.observe(stop_observer, names="value")
line_select.observe(line_observer, names="value")
destinations_select.observe(destination_observer, names="value")

Next passage in  0 days 00:05:00
